# <span style="color:#bcff8f"> Week 9 Assignment</span>

<span style="font-size:12pt;color:gray;font-weight:bold"> Patrick Weatherford</span><br>

<span style="font-size:16pt">Regression</span>

***
http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

***

<br>

## Importing packages

In [5]:
import os

# changing working directory to ThinkStats2/code folder
path = os.path.expanduser('~') + '\\OneDrive - Bellevue University\\Bellevue_University\\DSC 530 - Data Exploration and Analysis\\ThinkStats2\\code'
os.chdir(path)

%matplotlib inline

import thinkstats2 as ts2
import thinkplot as tp
import nsfg
import brfss
import random
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import bisect
import scipy as sp
from matplotlib.offsetbox import (AnchoredOffsetbox, TextArea)
import math
import first
from sklearn import linear_model as skl_lm
import pandas as pd
import statsmodels.formula.api as smf



<br><br>

## Custom Functions/Classes

In [9]:
def lm_var_mining(df, outcome_var, top_n=None):
    t = []

    for name in df.columns:
        try:
            if df[name].var() < 1e-7:  # if no variability then unreliable
                continue

            formula = f"{outcome_var} ~ {name}"
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:  # disregard variables where # of NULLs is greater than half the length of the dataframe
                continue

            results = model.fit()

        except (ValueError, TypeError):
            continue

        t.append((results.rsquared, name))


    t.sort(reverse=True)

    return t[:top_n]

<br><br>

## 11-1

Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.


In [7]:
live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]

In [11]:
## data mining to find relative variables
r_squared_var_list = lm_var_mining(df=live, outcome_var='prglngth')

## fit model to variables
model = smf.ols('prglngth ~ birthord==1 + race==2 + nbrnaliv>1', data=live)
results = model.fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     34.28
Date:                Tue, 08 Feb 2022   Prob (F-statistic):           5.09e-22
Time:                        22:04:24   Log-Likelihood:                -18247.
No. Observations:                8884   AIC:                         3.650e+04
Df Residuals:                    8880   BIC:                         3.653e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                38.7617      0.039   1006.410      0.000      38.686      38.837
birthord == 1[T.True]     0.1015      0.040      2.528      0.011       0.023       0.180
race == 2[T.True]         0.1390      0.042      3.311      0.001       0.057       0.221
nbrnaliv > 1[T.True]     -1.4944      0.164     -9.086      0.000      -1.817      -1.172
==============================================================================
Omnibus:                     1587.470   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6160.751
Skew:                          -0.852   Prob(JB):                         0.00
Kurtosis:                       6.707   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<br><br>

## 11-3

If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called poisson. It works the same way as ols and logit. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called numbabes. Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?


In [12]:
# Solution
live, firsts, others = first.MakeFrames()

resp = nsfg.ReadFemResp()
resp.index = resp.caseid

join = live.join(resp, on='caseid', rsuffix='_r')

# I used a nonlinear model of age.
join.numbabes.replace([97], np.nan, inplace=True)
join['age2'] = join.age_r**2

In [13]:

formula='numbabes ~ age_r + age2 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.678215
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 9148
Model:                        Poisson   Df Residuals:                     9141
Method:                           MLE   Df Model:                            6
Date:                Tue, 08 Feb 2022   Pseudo R-squ.:                 0.03632
Time:                        22:04:44   Log-Likelihood:                -15352.
converged:                       True   LL-Null:                       -15931.
Covariance Type:            nonrobust   LLR p-value:                9.041e-247
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.0197      0.166     -6.132      0.000      -1.346      -0.694
C(race)[T.2]    -0.1422      0.014     -9.827      0.000      -0.171      -0.114
C(race)[T.3]    -0.0980      0.024     -4.047      0.000      -0.146      -0.051
age_r            0.1544      0.010     15.157      0.000       0.134       0.174
age2            -0.0020      0.000    -13.230      0.000      -0.002      -0.002
totincr         -0.0186      0.002     -9.904      0.000      -0.022      -0.015
educat          -0.0464      0.003    -15.994      0.000      -0.052      -0.041
================================================================================
"""

In [14]:
## using model predictions for scenario
columns = ['age_r', 'age2', 'age3', 'race', 'totincr', 'educat']
new = pd.DataFrame([[35, 35**2, 35**3, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.507703
dtype: float64

<br><br>

## 11-4

If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called rmarital. Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?


In [15]:
# Solution

# Here's the best model I could find.
formula='rmarital ~ age_r + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.092083
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 9148
Model:                        MNLogit   Df Residuals:                     9113
Method:                           MLE   Df Model:                           30
Date:                Tue, 08 Feb 2022   Pseudo R-squ.:                  0.1661
Time:                        22:04:46   Log-Likelihood:                -9990.4
converged:                       True   LL-Null:                       -11981.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        8.9153      0.792     11.251      0.000       7.362      10.468
C(race)[T.2]    -0.9260      0.087    -10.705      0.000      -1.096      -0.756
C(race)[T.3]    -0.6335      0.133     -4.747      0.000      -0.895      -0.372
age_r           -0.3567      0.050     -7.132      0.000      -0.455      -0.259
age2             0.0047      0.001      6.054      0.000       0.003       0.006
totincr         -0.1301      0.011    -11.475      0.000      -0.152      -0.108
educat          -0.1940      0.018    -10.534      0.000      -0.230      -0.158
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        2.9927      2.970      1.007      0.314      -2.829       8.815
C(race)[T.2]    -0.3963      0.235     -1.685      0.092      -0.857       0.065
C(race)[T.3]     0.0650      0.336      0.194      0.846      -0.593       0.723
age_r           -0.3141      0.174     -1.806      0.071      -0.655       0.027
age2             0.0064      0.003      2.532      0.011       0.001       0.011
totincr         -0.3217      0.032    -10.135      0.000      -0.384      -0.259
educat          -0.1093      0.048     -2.266      0.023      -0.204      -0.015
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -3.6475      1.193     -3.059      0.002      -5.985      -1.310
C(race)[T.2]    -0.3303      0.091     -3.641      0.000      -0.508      -0.153
C(race)[T.3]    -0.8227      0.170     -4.853      0.000      -1.155      -0.490
age_r            0.1238      0.070      1.763      0.078      -0.014       0.261
age2            -0.0008      0.001     -0.814      0.416      -0.003       0.001
totincr         -0.2288      0.011    -20.058      0.000      -0.251      -0.206
educat           0.0661      0.016      4.015      0.000       0.034       0.098
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.3978      1.269     -1.889      0.059      -4.886       0.090
C(race)[T.2]    -1.0493      0.101    -10.366      0.000      -1.248      -0.851
C(race)[T.3]    -0.6065      0.154     -3.937      0.000      -0.908      -0.305
age_r            0.2084      0.077      2.699      0.007       0.057       0.360
age2            -0.0030      0.001     -2.619      0.009      -0.005      -0.001
totincr         -0.2900      0.014    -20.314      0.000      -0.318      -0.262
educat          -0.0176      0.021     -0.835      0.404      -0.059       0.024
-

Make a prediction for a woman who is 25 years old, white, and a high
school graduate whose annual household income is about $45,000.

In [16]:
# Solution

# This person has a 75% chance of being currently married, 
# a 13% chance of being "not married but living with opposite 
# sex partner", etc.

age = 25

columns = ['age_r', 'age2', 'race', 'totincr', 'educat']
new = pd.DataFrame([[age, age**2, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.745689,0.128291,0.001624,0.032801,0.021887,0.069708


<br><br>

***
***

## Chapter Practice

<span style="font-size:18px">$y=\beta_0+\beta_{i}x_{i}+e$</span>

<br>

$\beta_0=intercept$<br>
$\beta_i=parameter\ associated\ with\ x_i$<br>
$e=residual\ due\ to\ random\ variation$

<br>

<u>Simple regression</u>: 1 explanatory variable

<u>Multiple regression</u>: more than 1 explantory variable

Goal is to find parameters for all of the variables that minimize $e^2$.

In [17]:
import statsmodels.formula.api as smf

In [18]:
live, firsts, others = first.MakeFrames()

In [19]:
formula = 'totalwgt_lb ~ agepreg'
model = smf.ols(formula, data=live)  # ols stands for Ordinary Least Squares
results = model.fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     43.02
Date:                Tue, 08 Feb 2022   Prob (F-statistic):           5.72e-11
Time:                        22:04:47   Log-Likelihood:                -15897.
No. Observations:                9038   AIC:                         3.180e+04
Df Residuals:                    9036   BIC:                         3.181e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      6.8304      0.068    100.470      0.000       6.697       6.964
agepreg        0.0175      0.003      6.559      0.000       0.012       0.023
==============================================================================
Omnibus:                     1024.052   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3081.833
Skew:                          -0.601   Prob(JB):                         0.00
Kurtosis:                       5.596   Cond. No.                         118.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<br><br>

First babies tend to be lighter than others but is strange because there is no obvious mechanism that would cause first babies to be lighter. However, there is a possible explanation. Birth weight appears to be dependant on mother age and mothers age might be less when you're having your first child vs. subsequent children. Although obvious, we can test this theory.

In [20]:
## calculate weight means for firsts vs. others
diff_weight = firsts.agepreg.mean() - others.totalwgt_lb.mean()

## calculate age mean for firsts vs. others
diff_age = firsts.agepreg.mean() - others.agepreg.mean()

diff_weight, diff_age

(15.758558615710035, -3.586434766150152)

In [21]:
## regression analysis
results = smf.ols('totalwgt_lb ~ agepreg', data=live).fit()
slope = results.params['agepreg']
slope, slope*diff_age

(0.01745385147180278, -0.06259709972169449)

In [22]:
## multiple regression analysis
live['isfirst'] = live.birthord == 1
live['agepreg2'] = live.agepreg**2

formula = 'totalwgt_lb ~ isfirst + agepreg'
results = smf.ols(formula, data=live).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            totalwgt_lb   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     24.02
Date:                Tue, 08 Feb 2022   Prob (F-statistic):           3.95e-11
Time:                        22:04:48   Log-Likelihood:                -15894.
No. Observations:                9038   AIC:                         3.179e+04
Df Residuals:                    9035   BIC:                         3.182e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.9142      0.078     89.073      0.000       6.762       7.066
isfirst[T.True]    -0.0698      0.031     -2.236      0.025      -0.131      -0.009
agepreg             0.0154      0.003      5.499      0.000       0.010       0.021
==============================================================================
Omnibus:                     1019.945   Durbin-Watson:                   1.618
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3063.682
Skew:                          -0.599   Prob(JB):                         0.00
Kurtosis:                       5.588   Cond. No.                         137.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<br><br>

Data Mining for best variables

In [23]:
## join data together

live = live[live.prglngth > 30]

resp = nsfg.ReadFemResp()

resp.index = resp.caseid

join = live.join(resp, on='caseid', rsuffix='_r')

In [ ]:
t = []

for name in join.columns:
    try:
        if join[name].var() < 1e-7:  # if no variability then unreliable
            continue
            
        formula = f"totalwgt_lb ~ agepreg + {name}"
        model = smf.ols(formula, data=join)
        if model.nobs < len(join)/2:  # disregard variables where # of NULLs is greater than half the length of the dataframe
            continue
            
        results = model.fit()
            
    except (ValueError, TypeError):
        continue
        
    t.append((results.rsquared, name))
            
        
t.sort(reverse=True)

for mse, name in t[:30]:
    print(name, mse)

<br><br>

## Logistic Regression

Similar to linear regression but outcome is expressed in odds.

<span style="font-size:18px">$log(odds)=\beta_0+\beta_{i}x_{i}+e$</span>

In [ ]:
live, firsts, others = first.MakeFrames()

df1 = live[live.prglngth > 30]

df1['boy'] = (df1.loc[:]['babysex']==1).astype(int)

model = smf.logit('boy ~ agepreg',data=df1)
results = model.fit()

results.summary()


<br><br>

Model accuracy

In [ ]:
endog = pd.DataFrame(model.endog, columns=[model.endog_names])  # endog (aka endogenous) = outcome variable
exog = pd.DataFrame(model.exog, columns=[model.exog_names])  # exog (aka exogenous) = predictor variables

actual = endog['boy']
baseline = actual.mean()  # fraction of 1s vs 1s+0s

predict = (results.predict() >= 0.5)  # instances where prediction >= 0.5

true_pos = predict * actual
true_neg = (1-predict)*(1-actual)

acc = (sum(true_pos) + sum(true_neg) / len(actual))

acc